In [1]:
import ee,requests,os,sys,shutil
from IPython.display import Image
ee.Initialize(project='lcms-292214')

import geeViz.getImagesLib as gil
import geeViz.changeDetectionLib as cdl
Map = gil.Map

c:\Users\ihousman\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


geeViz package folder: c:\Users\ihousman\AppData\Local\Programs\Python\Python311\Lib\site-packages\geeViz


In [24]:
# Get CONUS Loss and Loss+Gain images

output_folder = '../assets/images'
version = '2023-9'
conus_loss = os.path.join(output_folder,f'CONUS_v{version}_Change_Loss.png')
conus_loss_gain = os.path.join(output_folder,f'CONUS_v{version}_Change_Loss_Gain.png')
conus_lc = os.path.join(output_folder,f'CONUS_v{version}_Land_Cover.png')
conus_lu = os.path.join(output_folder,f'CONUS_v{version}_Land_Use.png')
print(conus_loss)
gil.vizParamsTrue = {'min': 0.1, 'max': [0.3, 0.3, 0.3], 'bands': 'red,green,blue'}
sa = ee.FeatureCollection("projects/lcms-tcc-shared/assets/CONUS/Ancillary/Conus_NLCD_Boundary")

crs = gil.common_projections['NLCD_CONUS']['crs']
transform = gil.common_projections['NLCD_CONUS']['transform']
print(sa.geometry().bounds(500,crs).coordinates().getInfo())
dem = ee.Image("USGS/3DEP/10m").resample("bicubic")
hillshade = ee.Terrain.hillshade(dem)
# transform[0] = 300
# transform[-2] = 300
print(transform)

def getMostRecentChange(c, code):
    c = c.select(['Change'])
    def wrapper(img):
        yr = ee.Date(img.get("system:time_start")).get("year")
        out = ee.Image(yr).int16()
        out = out.where(img.neq(code),0).setDefaultProjection(crs,transform)#.selfMask()
        return (
            ee.Image(out
            .rename(["year"])
            .copyProperties(img, ["system:time_start"])).reduceResolution(ee.Reducer.firstNonNull(),True,256).selfMask()#.reproject(crs,transform)#.selfMask()
        )

    return c.map(wrapper)
def downloadImage(url,filename):
    img_data = requests.get(url).content
    with open(filename, 'wb') as handler:
        handler.write(img_data)

comp = ee.ImageCollection("projects/lcms-tcc-shared/assets/CONUS/Composites/Composite-Collection-yesL7")\
.filter(ee.Filter.calendarRange(2020,2023,'year')).mean()#.resample('bicubic').reproject(crs,transform)

comp = comp.visualize(**gil.vizParamsTrue10k)#.setDefaultProjection(crs,transform)#.reduceResolution(ee.Reducer.mean(),True).reproject(crs,transform)

lcms = ee.ImageCollection("USFS/GTAC/LCMS/v2023-9").filter("study_area=='CONUS'")
props = lcms.first().toDictionary()
lc = lcms.select(['Land_Cover']).sort('system:time_start',False).first().set(props).setDefaultProjection(crs,transform).reduceResolution(ee.Reducer.firstNonNull(),True,256).visualize()
lu = lcms.select(['Land_Use']).sort('system:time_start',False).first().set(props).setDefaultProjection(crs,transform).reduceResolution(ee.Reducer.firstNonNull(),True,256).visualize()

fast_loss = getMostRecentChange(lcms,3).mean().visualize(**{'min':1985,'max':2023,'palette':cdl.lossYearPalette})
slow_loss = getMostRecentChange(lcms,2).mean().visualize(**{'min':1985,'max':2023,'palette':cdl.lossYearPalette})
gain = getMostRecentChange(lcms,4).mean().visualize(**{'min':1985,'max':2023,'palette':cdl.gainYearPalette})

loss = comp
loss = loss.where(fast_loss.neq(0),fast_loss)
loss = loss.where(slow_loss.neq(0),slow_loss)

loss_gain = comp

loss_gain = loss_gain.where(fast_loss.neq(0),fast_loss)
loss_gain = loss_gain.where(slow_loss.neq(0),slow_loss)
loss_gain = loss_gain.where(gain.neq(0),gain)

# loss = hillshade.blend(loss.visualize(**{"opacity": 0.7}))
# loss_gain = hillshade.blend(loss_gain.visualize(**{"opacity": 0.7}))
Map.clearMap()
Map.addLayer(loss,{},'Loss')
Map.addLayer(loss_gain,{},'Loss+Gain')
Map.addLayer(lc,{},'Land Cover')
Map.addLayer(lu,{},'Land Use')
# Map.view()

params = {
  'region': sa.geometry().bounds(500,crs),
  'dimensions': 2000,
  'crs': crs,
  'transform':transform,
  'framesPerSecond': 3
}

# Get a URL that will generate a GIF.
# url = comps.getVideoThumbURL(params)
# loss_url = loss.getThumbURL(params)
# loss_gain_url = loss_gain.getThumbURL(params)
lc_url = lc.getThumbURL(params)
lu_url = lu.getThumbURL(params)

# downloadImage(loss_url,conus_loss)
# downloadImage(loss_gain_url,conus_loss_gain)
# https://apps.fs.usda.gov/fsgisx01/rest/services/RDW_LandscapeAndWildlife/LCMS_CONUS_Annual_Landcover/ImageServer/exportImage?f=image&bbox=-19716324.307481624%2C3034551.5331106214%2C-4688193.050393688%2C6243683.728634608&bboxSR=102100&imageSR=102100&size=1536%2C328&format=jpgpng&mosaicRule=%7B%22ascending%22%3Atrue%2C%22mosaicMethod%22%3A%22esriMosaicNorthwest%22%2C%22mosaicOperation%22%3A%22MT_FIRST%22%7D&renderingRule=%7B%22rasterFunction%22%3A%22Annual_Landcover%22%7D&time=1672444860000]\
downloadImage(lc_url,conus_lc)
downloadImage(lu_url,conus_lu)


../assets/images\CONUS_v2023-9_Change_Loss.png
[[[-2361585.0204712525, 259064.86686735478], [2263785.109653341, 259064.86686735478], [2263785.109653341, 3177404.865907416], [-2361585.0204712525, 3177404.865907416], [-2361585.0204712525, 259064.86686735478]]]
[30, 0, -2361915.0, 0, -30, 3177735.0]
Adding layer: Loss
Adding layer: Loss+Gain
Adding layer: Land Cover
Adding layer: Land Use


In [27]:
# Move gifs
from_dir = r'Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs'

which_ones = ['Atlanta-Land_Use_wYears.gif','Great_Salt_Lake-Land_Cover_wYears.gif','Northern_Colorado-Change_wYears.gif','Columbia_Glacier-Land_Cover_wYears.gif','Lake_Mead-Land_Cover_wYears.gif','Oregon_Wildfire-Change_wYears.gif','Las_Vegas-Land_Use_wYears.gif','Fountain_Wildfire-Change_wYears.gif','Denver-Land_Use_wYears.gif']

for w in which_ones:
    print('Copying:',os.path.basename(w))
    input = os.path.join(from_dir,os.path.basename(w))
    
    output = os.path.join(output_folder,os.path.basename(w))
    print(input,output)
    shutil.copy2(input,output)

Copying: Atlanta-Land_Use_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Atlanta-Land_Use_wYears.gif ../assets/images\Atlanta-Land_Use_wYears.gif
Copying: Great_Salt_Lake-Land_Cover_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Great_Salt_Lake-Land_Cover_wYears.gif ../assets/images\Great_Salt_Lake-Land_Cover_wYears.gif
Copying: Northern_Colorado-Change_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Northern_Colorado-Change_wYears.gif ../assets/images\Northern_Colorado-Change_wYears.gif
Copying: Columbia_Glacier-Land_Cover_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Columbia_Glacier-Land_Cover_wYears.gif ../assets/images\Columbia_Glacier-Land_Cover_wYears.gif
Copying: Lake_Mead-Land_Cover_wYears.gif
Z:\OutreachMaterials\Google\